# Sample Generator

## Preparation

In [ ]:
#create full screen

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import matplotlib.pyplot as ptl
from scipy.io import wavfile

In [ ]:
#name for notes
note_names = ['A_m', 'Ash_m', 'B_m', 'C', 'Csh', 'D', 'Dsh', 'E', 'F', 'Fsh', 'G', 'Gsh', 'A', 'Ash', 'B', 'C_p', 'Csh_p', 'D_p', 'Dsh_p', 'E_p']

#frequency for notes
note_frequency_Hz = [220, 233.08, 246.94, 261.63, 277.18, 293.67, 311.13, 329.63, 349.23, 369.99, 392, 418.3, 440, 466.16, 493.88, 523.25, 554.36, 587.33, 622.26, 659.26]

#name for styles
# sin - sine wave
# mndr - square wave
# trin - sowtool wave
# gitr - gitar samples
note_form_style = ['sin', 'mndr', 'trin', 'gitr']

#coefficient for the DOR register in a DAC.
# DOR_K = 4096 / V_ref = 4096 / 3.3 = 1241.212121
DOR_K = 1241.212121

#sampling period (48000 - sampling frequency)
time_step = 1/48000

#time_step for exponent
exp_step = 0.007

In [ ]:
#read the gitar sample
samplerate, data = wavfile.read('./sampl1.wav')

#gitar sample with interpolation
interpolation_data = []
old_sempl = 0

#interpolation the gitar sample  
for i in data:
    full_sptep = i - old_sempl
    step = round(full_sptep / 10)
    if (step != 0):
        interpolation_data += (list(range(old_sempl, i, step)))
    else:
        interpolation_data += ([i for x in range(1, 10)])
    old_sempl = i

In [ ]:
ptl.plot(interpolation_data)

## Exponent calculation

In [ ]:
def calc_exp_form (teme_step):
    
    '''function to calculate the exponent'''
    
    note_form = []
    for t in range(0, 500):
        note_form.append(round((1 / np.exp(teme_step * t)), 5))
    return note_form

In [ ]:
#print array with the exponent in C-format

exp_fr = calc_exp_form(exp_step)

print('const float EXPONENT [500] = {', end = '')

for i, exp_s in enumerate(exp_fr):
    temp_str = str(exp_s)
    
    spases = ',' + ' ' * (8 - len(temp_str))
    
    if ((i % 14) == 13):
        spases = ',\n' + ' ' * 30
    
    if (i == len(exp_fr)-1):
        print(temp_str, end='};')
    else:
        print(temp_str, end=spases)

In [ ]:
ptl.plot(exp_fr)

## Calculating the sine for vibrato

In [ ]:
def calc_vibrato (note_freq, teme_step):
    
    '''function to calculate the sine for vibrato'''
    
    note_form = []
    new_val = 0
    old_val = 0
    for t in np.arange(0, 1, time_step):
        new_val = np.sin(np.pi * 2 * note_freq * t)
        if (new_val >= 0 and old_val < 0):
            return note_form
        else:
            note_form.append(new_val)
            old_val = new_val
    

In [ ]:
#print array with the sine for vibrato in C-format

vibrato_fr = (calc_vibrato(100, time_step)[:-1])
vibrato_fr = np.multiply(vibrato_fr, 0.15)
vibrato_fr = np.add(vibrato_fr, 0.8)

print('const float vibrato [' + str(len(vibrato_fr)) + '] =  {', end = '')

for i, vibrato_s in enumerate(vibrato_fr):
    temp_str = str(round(vibrato_s, 5))
    
    spases = ',' + ' ' * (8 - len(temp_str))
    
    if ((i % 14) == 13):
        spases = ',\n' + ' ' * 30
    
    if (i == len(vibrato_fr)-1):
        print(temp_str, end='};')
    else:
        print(temp_str, end=spases)

In [ ]:
ptl.plot(vibrato_fr)

## Calculating the sine wave

In [ ]:
def print_form_note(note_name, note_form, note_form_style, size_tabs = 40, end = '\n\n'):
    
    '''function to print array in C-format'''
    
    data_type = "const int16_t "
    define_form_string = data_type + note_name + '_form_' + note_form_style + ' [' + str(len(note_form)) + '] ='
    temp = len(define_form_string)
    define_form_string += ' ' * (size_tabs - temp - 1) + '{'
    
    print(define_form_string, end = '')
    
    for i, n in enumerate(note_form):
        if (n >= 0):
            if (n < 10):
                spases = ',    '
            elif (n < 100):
                spases = ',   '
            else:
                spases = ',  '
        else:
            if (n > -10):
                spases = ',   '
            elif (n > -100):
                spases = ',  '
            else:
                spases = ', '

        if ((i % 20) == 19):
            spases = ',\n' + ' ' * size_tabs

        if (i == len(note_form)-1):
            print(n, end='};')
        else:
            print(n, end=spases)
    
    print(end, end = '')

In [ ]:
def calc_note_form_sin (note_mug, note_freq, DOR_K, teme_step):
    
    '''function to calculate the sine form for notes'''
    
    note_form = []
    new_val = 0
    old_val = 0
    for t in np.arange(0, 0.01, time_step):
        new_val = round(note_mug * np.sin(np.pi * 2 * note_freq * t) * DOR_K)
        
        if (new_val >= 0 and old_val < 0):
            return note_form
        else:
            note_form.append(new_val)
            old_val = new_val

In [ ]:
#print array with the sine wave in C-format

note_magnitude = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
sizes = []
sin_nt_fm = []

for n, m, f in zip(note_names, note_magnitude, note_frequency_Hz):
    sin_nt_fm = calc_note_form_sin(m, f, DOR_K, time_step)
    sizes.append(len(sin_nt_fm))
    print_form_note(n, sin_nt_fm, note_form_style[0])
    
    
print('const int16_t PAUSE_form_sin [30] = {0};' ,end = '\n\n')
    


for i, n in enumerate(note_names):
    if (i == 0):
        print('const int16_t *note_sin_table[21] =    {' + n + '_form_' + note_form_style[0] + ',')
    else:
        print(' ' * 40 + n + '_form_' + note_form_style[0] + ',')
        
print(' ' * 40 + 'PAUSE_form_sin};', end = '\n\n')

sizes.append(30)

temp_str = str(sizes)
temp_str = temp_str.replace('[', '{')
temp_str = temp_str.replace(']', '}')
print('uint16_t note_form_' + note_form_style[0] +'_size[21] = ' + temp_str + ';')

In [ ]:
ptl.plot(sin_nt_fm)

## Calculating the square wave

In [ ]:
def calc_note_form_mndr (note_mug, note_freq, DOR_K, teme_step):
    
    '''function to calculate the square form for notes'''
    
    note_form = []
    new_val = 0
    old_val = 0
    for t in np.arange(0, 0.01, time_step):
        new_val = round(note_mug * (1 if np.sin(np.pi * 2 * note_freq * t) >= 0 else -1) * DOR_K)
        if (new_val >= 0 and old_val < 0):
            return note_form
        else:
            note_form.append(new_val)
            old_val = new_val

In [ ]:
#print array with the square wave in C-format

note_magnitude = [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15]
sizes = []
mndr_nt_fm = []

for n, m, f in zip(note_names, note_magnitude, note_frequency_Hz):
    mndr_nt_fm = calc_note_form_mndr(m, f, DOR_K, time_step)
    sizes.append(len(mndr_nt_fm))
    print_form_note(n, mndr_nt_fm, note_form_style[1])
    
    
print('const int16_t PAUSE_form_mndr [30] = {0};' ,end = '\n\n')
    

for i, n in enumerate(note_names):
    if (i == 0):
        print('const int16_t *note_mndr_table[21] =   {' + n + '_form_' + note_form_style[1] + ',')
    else:
        print(' ' * 40 + n + '_form_' + note_form_style[1] + ',')
        
print(' ' * 40 + 'PAUSE_form_mndr};', end = '\n\n')

sizes.append(30)

temp_str = str(sizes)
temp_str = temp_str.replace('[', '{')
temp_str = temp_str.replace(']', '}')
print('uint16_t note_form_' + note_form_style[1] +'_size[21] = ' + temp_str + ';')


In [ ]:
ptl.plot(mndr_nt_fm)

## Calculating the sowtool wave

In [ ]:
def calc_note_form_trn(note_mug, note_freq, bs, DOR_K, teme_step):
    
    '''function to calculate the sowtool form for notes'''
    
    trin_form = np.arange(-1, 1, 2/bs)

    trin_form = trin_form * note_mug * DOR_K
    
    trin_form = trin_form[0: bs]
    
    return (np.around(trin_form)).astype(int)


In [ ]:
#print array with the sowtool wave in C-format

note_magnitude = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
buf_sizes = [219, 206, 195, 184, 174, 164, 155, 146, 138, 130, 123, 115, 110, 103, 98, 92, 87, 82, 78, 73, 30]
sizes = []
trin_nt_fm = []

for n, m, f, bs in zip(note_names, note_magnitude, note_frequency_Hz, buf_sizes):
    trin_nt_fm = calc_note_form_trn(m, f, bs, DOR_K, time_step)
    sizes.append(len(trin_nt_fm))
    print_form_note(n, trin_nt_fm, note_form_style[2])
    
    
print('const int16_t PAUSE_form_trin [30] = {0};' ,end = '\n\n')
    


for i, n in enumerate(note_names):
    if (i == 0):
        print('const int16_t *note_trin_table[21] =   {' + n + '_form_' + note_form_style[2] + ',')
    else:
        print(' ' * 40 + n + '_form_' + note_form_style[2] + ',')
        
print(' ' * 40 + 'PAUSE_form_trin};', end = '\n\n')

sizes.append(30)

temp_str = str(sizes)
temp_str = temp_str.replace('[', '{')
temp_str = temp_str.replace(']', '}')
print('uint16_t note_form_' + note_form_style[2] +'_size[21] = ' + temp_str + ';')


In [ ]:
ptl.plot(trin_nt_fm)

## Calculating the gitar samples

In [ ]:
def gitar_sampl(note_mug, sampl_form, note_sampl_size):
    
    '''function to calculate the gitar sample for notes'''
    
    note_form = []
    scale_val = max(sampl_form) / note_mug 
    
    for i in range(0, note_sampl_size):
        sampl = sampl_form[i*int(len(sampl_form)/note_sampl_size)]
        note_form.append(int(round(sampl / scale_val)))
    
    return note_form

In [ ]:
#print array with the gitar sample in C-format

note_magnitude = [300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300]
buf_sizes = [219, 206, 195, 184, 174, 164, 155, 146, 138, 130, 123, 115, 110, 103, 98, 92, 87, 82, 78, 73, 30]
sizes = []
gitr_nt_fm = []

for n, m, f, bs in zip(note_names, note_magnitude, note_frequency_Hz, buf_sizes):
    gitr_nt_fm = gitar_sampl(m, interpolation_data, bs)
    sizes.append(len(gitr_nt_fm))
    print_form_note(n, gitr_nt_fm, note_form_style[3])
    
    
print('const int16_t PAUSE_form_gitr [30] = {0};' ,end = '\n\n')
    


for i, n in enumerate(note_names):
    if (i == 0):
        print('const int16_t *note_gitr_table[21] =   {' + n + '_form_' + note_form_style[3] + ',')
    else:
        print(' ' * 40 + n + '_form_' + note_form_style[3] + ',')
        
print(' ' * 40 + 'PAUSE_form_gitr};', end = '\n\n')

sizes.append(30)

temp_str = str(sizes)
temp_str = temp_str.replace('[', '{')
temp_str = temp_str.replace(']', '}')
print('uint16_t note_form_' + note_form_style[3] + '_size[21] = ' + temp_str + ';')

In [ ]:
ptl.plot(gitr_nt_fm)